# Shor's algorithm for computing discrete logarithms
This notebook exemplifies using [Quaspy](https://github.com/ekera/quaspy) to simulate Shor's algorithm for computing general discrete logarithms [[Shor94]](https://doi.org/10.1109/SFCS.1994.365700), modified as in [[E19p]](https://doi.org/10.48550/arXiv.1905.09084), and with the classical post-processing from [[E19p]](https://doi.org/10.48550/arXiv.1905.09084).

To start off, let us define the P-384 curve $E$ and associated generator $g$ of order $r$ as specified in [NIST SP 800-186](https://doi.org/10.6028/NIST.SP.800-186).

In [1]:
!pip3 install -q --pre quaspy # Make sure that quaspy is installed.

In [2]:
from quaspy.math.groups import ShortWeierstrassCurveOverPrimeField;
from quaspy.math.groups import PointOnShortWeierstrassCurveOverPrimeField;

# Define the elliptic curve.
p = 39402006196394479212279040100143613805079739270465446667948293404245721771496870329047266088258938001861606973112319;

a = -3;
b = 27580193559959705877849011840389048093056905856361568521428707301988689241309860865136260764883745107765439761230575;

E = ShortWeierstrassCurveOverPrimeField(a, b, p);

# Define the generator.
g_x = 26247035095799689268623156744566981891852923491109213387815615900925518854738050089022388053975719786650872476732087;
g_y = 8325710961489029985546751289520108179287853048861315594709205902480503199884419224438643760392947333078086511627871;

g = PointOnShortWeierstrassCurveOverPrimeField(g_x, g_y, E);

# Define the order of the generator.
r = 39402006196394479212279040100143613805079739270465446667946905279627659399113263569398956308152294913554433653942643;

Let us continue to sample the exponent $d$ from $[0, r) \cap \mathbb Z$.

To this end, we use the [<code>sample_integer(B)</code>](../docs/math/random/sample_integer.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy).

In [3]:
from quaspy.math.random import sample_integer;

# Sample d.
m = 384;
d = sample_integer(r);

print("Sampled d =", d);

# Compute x.
x = g ** d;

print("\nComputed x =", x);

Sampled d = 35773185701206256782389453125645907362921156418776666313236175287270064070982806501651049954067133099835270458090283

Computed x = (26839090637848134892919745247034038406278373958672665401616852826868571713177666001845223681473060814587388838646401, 25182122108607584919760546446572336973082854037194233293533787963866589372314610230855156176325809247998651354088087) on y^2 = x^3 + 39402006196394479212279040100143613805079739270465446667948293404245721771496870329047266088258938001861606973112316 x + 27580193559959705877849011840389048093056905856361568521428707301988689241309860865136260764883745107765439761230575 (mod 39402006196394479212279040100143613805079739270465446667948293404245721771496870329047266088258938001861606973112319)


## 1. Solving for the logarithm $d$ given $r$ in a single run
To start off, let us first consider the setting where our goal is to solve for the discrete logarithm $d$ given the order $r$ in a single run of the quantum part of Shor's algorithm [[Shor94]](https://doi.org/10.1109/SFCS.1994.365700) modified as in [[E19p]](https://doi.org/10.48550/arXiv.1905.09084), and with the classical post-processing from [[E19p]](https://doi.org/10.48550/arXiv.1905.09084).

### 1.1. Sampling a frequency pair $(j, k)$
[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_j_k_given_d_r_heuristic(d, r, m, sigma, l, ..)</code>](../docs/logarithmfinding/sampling/sample_j_k_given_d_r_heuristic.md) for heuristically simulating the quantum part of Shor's algorithm modified as in [[E19p]](https://doi.org/10.48550/arXiv.1905.09084) for a given logarithm $d$, order $r$, and parameters $m$, $\varsigma$ and $\ell$, where $m$ is an upper bound on the bit length of $r$, $\varsigma$ is a non-negative integer and $\ell$ is a positive integer. For further details, see [[E19p]](https://doi.org/10.48550/arXiv.1905.09084).

Below, we use said function to simulate running the algorithm for $d$ and $r$ with control registers of length $m + \varsigma$ qubits and $\ell$ qubits respectively.
More specifically, for $g$ a generator and $x = g^d$, we simulate inducing the state

$$\frac{1}{2^{m + \varsigma + \ell}}
\sum_{a, \, j \, = \, 0}^{2^{m+\varsigma} - 1}
\sum_{b, \, k \, = \, 0}^{2^{\ell} - 1}
\mathrm{exp}
\left(
  \frac{2 \pi \mathrm{i}}{2^{m + \varsigma}} (aj + 2^{m+\varsigma-\ell} bk)
\right)
|\, j, k, g^a x^{-b} \,\rangle$$

and reading out the first two control registers.
This yields a frequency pair $(j, k)$ sampled from the probability distribution induced by the quantum part of the algorithm:

In [4]:
from quaspy.logarithmfinding.sampling import sample_j_k_given_d_r_heuristic;

sigma = 0;
l = m;

[j, k] = sample_j_k_given_d_r_heuristic(d, r, m, sigma, l, timeout = 30);

print("Sampled j =", j);
print("Sampled k =", k);

Sampled j = 22160468565457300365530738477763505403106646292489537185757901985106926272610027630708214989346589022038800714152928
Sampled k = 28049371525036017893479770229003917987755673458228553796757902524485196773199942341459745367679150254785327158797765


As explained in [[E19p]](https://doi.org/10.48550/arXiv.1905.09084), $\sim \ell$ bits of information on $d$ is computed in each run of the quantum part of Ekerå–Håstad's algorithm. To solve in a single run, we therefore select $\ell = m$ above. By increasing $\varsigma$, additional peaks that arise due to noise can be suppressed. When solving in a single run, we can afford to seek over the different peaks in the classical post-processing, and we therefore select $\varsigma = 0$ above.

### 1.2. Solving the frequency pair $(j, k)$ and $g$ and $x$ for $d$ given $r$
We now proceed to solve the frequency pair $(j, k)$ for the logarithm $d$ given the order $r$.

To this end, we use the [<code>solve_j_k_for_d_given_r(j, k, m, sigma, l, g, x, r, ..)</code>](../docs/logarithmfinding/general/postprocessing/solve_j_k_for_d_given_r.md) function provided by [Quaspy](https://github.com/ekera/quaspy). 
It solves $(j, k)$ for $d$ given $r$ by using the lattice-based post-processing from [[E19p]](https://doi.org/10.48550/arXiv.1905.09084).

In [5]:
from quaspy.logarithmfinding.general.postprocessing import solve_j_k_for_d_given_r;

recovered_d = solve_j_k_for_d_given_r(j, k, m, sigma, l, g, x, r);

if (recovered_d == d):
  print("Recovered d =", d);

  print("\n[ OK ] Successfully recovered d.");
else:
  print("[FAIL] Failed to recover d.");

Recovered d = 35773185701206256782389453125645907362921156418776666313236175287270064070982806501651049954067133099835270458090283

[ OK ] Successfully recovered d.


## 2. Making tradeoffs and solving for $d$ given $r$ in multiple runs
Let us now consider the case where we make tradeoffs by picking $\ell \approx m/s$ for $s$ some tradeoff factor.

As explained in [[E19p]](https://doi.org/10.48550/arXiv.1905.09084) and [[E24t]](https://diva-portal.org/smash/get/diva2:1902626/FULLTEXT01.pdf) (see Sect. 5.5), each run of the quantum part of Shor's algorithm modified as in [[E19p]](https://doi.org/10.48550/arXiv.1905.09084) heuristically yields $\sim \ell$ bits of information on the logarithm $d$.
Hence, we heuristically expect to have to perform at least $s$ runs to solve for $d$ given $r$ efficiently and with high probability of success in the classical post-processing.

According to the heuristic estimates in [[E24t]](https://diva-portal.org/smash/get/diva2:1902626/FULLTEXT01.pdf) (see Tabs. 5.4 and 5.5 on p. 80) which were computed with the [Qunundrum](https://github.com/ekera/qunundrum) suite of MPI programs, when $m = 384$, $s = 8$ and $\ell = \lceil m / s \rceil$, we heuristically need to make no more than $n = 11$ runs to solve efficiently in the classical post-processing with $\ge 99\%$ success probability without enumerating the lattice.
In the below example, we use this specific parameterization.

### 2.1. Sampling $n$ frequency pairs $((j_1, k_1), \, \ldots, \, (j_n, k_n))$
To start off, in analogy with Sect. 1.1 above, we heuristically sample $n$ frequency pairs $((j_1, k_1), \, \ldots, \, (j_n, k_n))$ from the distribution induced by the quantum part of Shor's algorithm modified as in [[E19p]](https://doi.org/10.48550/arXiv.1905.09084).
To this end, we use the [<code>sample_j_k_given_d_r_heuristic(d, r, m, sigma, l, ..)</code>](../docs/logarithmfinding/sampling/sample_j_k_given_d_r_heuristic.md) function provided by [Quaspy](https://github.com/ekera/quaspy).

In [6]:
from quaspy.logarithmfinding.sampling import sample_j_k_given_d_r_heuristic;

from math import ceil;

sigma = 9
s = 8;
l = ceil(m / s);
n = 11;

j_k_list = [sample_j_k_given_d_r_heuristic(d, r, m, sigma, l, timeout = 30) \
              for _ in range(n)];

print("Sampled pairs [j, k] =", j_k_list);

Sampled pairs [j, k] = [[18757216399025630899805659090112177406425803989145763857464119548338470559017736742990465994109342835012720966546169303, 216059640135317], [9632536199203028041114865368697085517542763140750805902830058197952491920127375611242357472609185260670886501392387405, 206255684141109], [17691928455994243274382382105213728079278935449833919961274332010131985864495169624256102236134168405640785439899135316, 233785455308049], [18401735677033509643472235240560404686962988461478841416456751209944650986917214831044416052923390442811058692912683951, 227712613552169], [17004693260574759854061823525182608771363442695896929485703780451156201712568243791007691733771793454902720767594685689, 190976820074786], [1908265761582526492702045721953824201192805902151196435640887269316059339929522213449412786503988328434649492023152159, 253993219320439], [1839854491009785738627560939472314180836260119589096352886845629978518103848484244205131695285550512183510115059212486, 190472318355106],

### 2.2. Solving the $n$ frequency pairs $((j_1, k_1), \, \ldots, \, (j_n, k_n))$ and $g$ and $x$ for $d$ given $r$

We now proceed to solve the $n$ frequency pairs $((j_1, k_1), \, \ldots, \, (j_n, k_n))$ for the logarithm $d$ given the order $r$.

To this end, we use the [<code>solve_multiple_j_k_for_d_given_r(j_k_list, m, l, g, x, ..)</code>](../docs/logarithmfinding/short/postprocessing/solve_multiple_j_k_for_d.md) function provided by [Quaspy](https://github.com/ekera/quaspy).
It jointly solves  $((j_1, k_1), \, \ldots, \, (j_n, k_n))$ for $d$ by using the lattice-based post-processing from [[E19p]](https://doi.org/10.48550/arXiv.1905.09084).

In [7]:
from quaspy.logarithmfinding.general.postprocessing import solve_multiple_j_k_for_d_given_r;

recovered_d = solve_multiple_j_k_for_d_given_r(j_k_list,
                                               m,
                                               sigma,
                                               l,
                                               g,
                                               x,
                                               r,
                                               enumerate = False,
                                               timeout = 30);

if (recovered_d == d):
  print("Recovered d =", d);

  print("\n[ OK ] Successfully recovered d.");
else:
  print("[FAIL] Failed to recover d.");

Recovered d = 35773185701206256782389453125645907362921156418776666313236175287270064070982806501651049954067133099835270458090283

[ OK ] Successfully recovered d.
